<a href="https://colab.research.google.com/github/Khushi04092004/LicenseDetectorModel/blob/main/LicensePlateDetector_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install ultralytics easyocr opencv-python numpy matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 842.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import os
import cv2
import random
import shutil
from tqdm import tqdm

# Set paths
video_dir = "/content/drive/MyDrive/NumberPlateVideos"  # folder where your videos are
label_dir = "/content/drive/MyDrive/VideoLabels"  # YOLO .txt labels for each frame
output_dir = "/content/drive/MyDrive/VideoDataset"  # where you want images/labels split

# Create output directories
os.makedirs(output_dir, exist_ok=True)
for split in ["train", "val"]:
    os.makedirs(f"{output_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{split}", exist_ok=True)

# Step 1: Extract frames from all videos
frame_output_dir = f"{output_dir}/frames"
os.makedirs(frame_output_dir, exist_ok=True)

frame_count = 0
video_files = [f for f in os.listdir(video_dir) if f.endswith(".mp4")]
print(f"Found {len(video_files)} video files")

for video_file in tqdm(video_files, desc="Processing videos"):
    video_path = os.path.join(video_dir, video_file)
    video_name = os.path.splitext(video_file)[0]

    # Extract frames at 1 frame per second to avoid too many similar frames
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(fps)  # Extract 1 frame per second

    if not cap.isOpened():
        print(f"Warning: Could not open video: {video_path}")
        continue

    video_frame_count = 0
    frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract only every nth frame based on the interval
        if frame_index % frame_interval == 0:
            frame_name = f"{video_name}_{video_frame_count:05d}.jpg"
            frame_path = os.path.join(frame_output_dir, frame_name)
            cv2.imwrite(frame_path, frame)
            video_frame_count += 1
            frame_count += 1

        frame_index += 1

    cap.release()
    print(f"Extracted {video_frame_count} frames from {video_file}")

print(f"✅ Extracted total {frame_count} frames from all videos.")

# Check if frames were actually extracted
if frame_count == 0:
    print("❌ No frames were extracted. Check video files and paths.")
    exit()

# Step 2: Split frames into train/val (80-20)
all_frames = [f for f in os.listdir(frame_output_dir) if f.endswith(".jpg")]
print(f"Found {len(all_frames)} frames in output directory")

if len(all_frames) == 0:
    print("❌ No frames found in output directory. Check extraction process.")
    exit()

random.shuffle(all_frames)

split_ratio = 0.8
split_index = int(len(all_frames) * split_ratio)

train_frames = all_frames[:split_index]
val_frames = all_frames[split_index:]

print(f"Splitting into {len(train_frames)} training frames and {len(val_frames)} validation frames")

# Helper to copy frames and labels with improved debugging
def move_data(frames, split):
    copied_images = 0
    copied_labels = 0
    missing_labels = 0

    for frame in tqdm(frames, desc=f"Copying {split} data"):
        # Copy image
        src_img = os.path.join(frame_output_dir, frame)
        dst_img = os.path.join(output_dir, "images", split, frame)

        if os.path.exists(src_img):
            shutil.copy(src_img, dst_img)
            copied_images += 1
        else:
            print(f"Warning: Source image not found: {src_img}")
            continue

        # Copy matching label
        label_name = frame.replace(".jpg", ".txt")
        src_lbl = os.path.join(label_dir, label_name)
        dst_lbl = os.path.join(output_dir, "labels", split, label_name)

        if os.path.exists(src_lbl):
            shutil.copy(src_lbl, dst_lbl)
            copied_labels += 1
        else:
            missing_labels += 1
            # Uncomment the next line if you want to see all missing labels
            # print(f"Warning: Label not found for: {label_name}")

    print(f"✅ {split} set: Copied {copied_images} images and {copied_labels} labels ({missing_labels} missing labels)")
    return copied_images, copied_labels

# Move both train and val sets
train_stats = move_data(train_frames, "train")
val_stats = move_data(val_frames, "val")

# Verify the dataset structure
print("\nDataset Statistics:")
print(f"Training set: {train_stats[0]} images, {train_stats[1]} labels")
print(f"Validation set: {val_stats[0]} images, {val_stats[1]} labels")

# Verify files exist in the destination directories
train_img_dir = os.path.join(output_dir, "images", "train")
val_img_dir = os.path.join(output_dir, "images", "val")
train_lbl_dir = os.path.join(output_dir, "labels", "train")
val_lbl_dir = os.path.join(output_dir, "labels", "val")

print(f"\nFiles in directories:")
print(f"- Training images: {len(os.listdir(train_img_dir))}")
print(f"- Validation images: {len(os.listdir(val_img_dir))}")
print(f"- Training labels: {len(os.listdir(train_lbl_dir))}")
print(f"- Validation labels: {len(os.listdir(val_lbl_dir))}")

print("\n✅ Dataset preparation completed!")

Found 8 video files


Processing videos:  12%|█▎        | 1/8 [00:06<00:43,  6.24s/it]

Extracted 31 frames from Automatic Number Plate Recognition (ANPR) _ Vehicle Number Plate Recognition (1).mp4


Processing videos:  25%|██▌       | 2/8 [02:10<07:35, 75.90s/it]

Extracted 60 frames from Traffic Control CCTV.mp4


...

Extracted 20 frames from pexels-taryn-elliott-5309381 (1080p).mp4
✅ Extracted total 229 frames from all videos.
Found 229 frames in output directory
Splitting into 183 training frames and 46 validation frames


Copying train data: 100%|██████████| 183/183 [00:07<00:00, 22.88it/s]


✅ train set: Copied 183 images and 0 labels (183 missing labels)


Copying val data: 100%|██████████| 46/46 [00:01<00:00, 28.40it/s]

✅ val set: Copied 46 images and 0 labels (46 missing labels)

Dataset Statistics:
Training set: 183 images, 0 labels
Validation set: 46 images, 0 labels

Files in directories:
- Training images: 183
- Validation images: 46
- Training labels: 0
- Validation labels: 0

✅ Dataset preparation completed!


In [1]:
# Install YOLOv5 and other required packages
!pip install torch torchvision torchaudio
!pip install ultralytics
!pip install easyocr
!pip install pandas matplotlib seaborn
!pip install opencv-python-headless
!pip install tqdm
!pip install scikit-learn

# Clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt
%cd ..

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# Import necessary libraries
import os
import cv2
import numpy as np
from google.colab import drive

# Mount Google Drive
try:
    drive.mount('/content/drive')
    print("Drive mounted successfully")
except:
    print("Running in local environment, skipping drive mount")

# Install necessary packages (without roboflow)
!pip install -q opencv-python-headless ultralytics

# Set the paths
base_dir = "/content/drive/MyDrive/VideoDataset"
images_train_path = f"{base_dir}/images/train"
images_val_path = f"{base_dir}/images/val"
labels_train_path = f"{base_dir}/labels/train"
labels_val_path = f"{base_dir}/labels/val"

# Create directories if they don't exist
for dir_path in [labels_train_path, labels_val_path]:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# Method 1: Automated labeling using a pre-trained model
from ultralytics import YOLO

print("Loading pre-trained model for license plate detection...")
# Use YOLOv8 to detect vehicles first
model = YOLO('yolov8n.pt')

# Function to process images and generate labels
def create_labels(images_dir, labels_dir):
    """Generate YOLO format labels for images

    Args:
        images_dir: Directory containing images
        labels_dir: Directory to save labels
    """
    if not os.path.exists(images_dir):
        print(f"Error: Images directory {images_dir} not found!")
        return

    files_processed = 0

    # Process each image
    for img_file in os.listdir(images_dir):
        if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
            # Load image
            img_path = os.path.join(images_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Failed to load image: {img_path}")
                continue

            # Get predictions
            results = model(img)

            # Create label file name
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(labels_dir, label_file)

            vehicles_found = False

            # Open label file for writing
            with open(label_path, 'w') as f:
                # For each detection
                for box in results[0].boxes:
                    cls = int(box.cls[0])
                    conf = float(box.conf[0])

                    # Filter for vehicles (car=2, truck=7, bus=5 in COCO)
                    if cls in [2, 5, 7] and conf > 0.25:
                        vehicles_found = True
                        # Convert bbox to YOLO format (x_center, y_center, width, height)
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        h, w = img.shape[:2]

                        # Convert to normalized YOLO format
                        x_center = ((x1 + x2) / 2) / w
                        y_center = ((y1 + y2) / 2) / h
                        width = (x2 - x1) / w
                        height = (y2 - y1) / h

                        # For license plates, use class 0
                        f.write(f"0 {x_center} {y_center} {width} {height}\n")

            # If no vehicles found, create an empty label file with a small dummy box
            # This ensures we don't have missing label files
            if not vehicles_found:
                with open(label_path, 'w') as f:
                    # Creating a small dummy box in the center (this is just to avoid errors)
                    # You might want to delete or annotate these files later
                    f.write("0 0.5 0.5 0.1 0.1\n")

            files_processed += 1
            if files_processed % 50 == 0:
                print(f"Processed {files_processed} images...")

    print(f"Completed processing {files_processed} images in {images_dir}")

# Create labels for training and validation sets
print("Creating labels for training images...")
create_labels(images_train_path, labels_train_path)

print("Creating labels for validation images...")
create_labels(images_val_path, labels_val_path)

# Create YAML configuration file
yaml_content = """# License Plate Detection Dataset
train: {0}/images/train/
val: {0}/images/val/

# Number of classes
nc: 1

# Class names
names: ['license_plate']
""".format(base_dir)

yaml_path = "license_plate_data.yaml"
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"Created dataset configuration file: {yaml_path}")

# Clone YOLOv5 if it doesn't exist
if not os.path.exists("yolov5"):
    !git clone https://github.com/ultralytics/yolov5
    !cd yolov5 && pip install -r requirements.txt

# Start training
print("\n=== Ready to train! ===")
print("Run the following command to start training:")
print(f"!cd yolov5 && python train.py --img 640 --batch 16 --epochs 100 --data ../{yaml_path} --weights yolov5s.pt")

# Uncomment the following line to start training immediately
# !cd yolov5 && python train.py --img 640 --batch 16 --epochs 100 --data ../license_plate_data.yaml --weights yolov5s.pt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted successfully
Created directory: /content/drive/MyDrive/VideoDataset/labels/train
Created directory: /content/drive/MyDrive/VideoDataset/labels/val
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading pre-trained model for license plate detection...


100%|██████████| 6.25M/6.25M [00:00<00:00, 407MB/s]


Creating labels for training images...

0: 384x640 1 person, 25 cars, 1 truck, 84.4ms
Speed: 17.8ms preprocess, 84.4ms inference, 394.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 8 motorcycles, 1 truck, 17.6ms
Speed: 5.4ms preprocess, 17.6ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 7.6ms
Speed: 3.5ms preprocess, 7.6ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 cars, 7.0ms
Speed: 2.9ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 640x384 12 cars, 1 truck, 43.5ms
Speed: 3.9ms preprocess, 43.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 384x640 6 persons, 5 cars, 2 buss, 8.0ms
Speed: 3.7ms preprocess, 8.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 3 cars, 1 truck, 10.8ms
Speed: 3.3ms preprocess, 10.8ms inference, 1.3ms postprocess per image at shape 

In [4]:
# Create license_plate_data.yaml file
yaml_content = """
# Dataset paths
train: /content/drive/MyDrive/VideoDataset/images/train/
val: /content/drive/MyDrive/VideoDataset/images/val/

# Number of classes
nc: 1

# Class names
names: ['license_plate']
"""

with open('license_plate_data.yaml', 'w') as f:
    f.write(yaml_content)

print("Created YAML file for dataset configuration")

Created YAML file for dataset configuration


In [5]:
# First, let's reduce memory usage by modifying training parameters
import os
import gc
import torch

# Clear cache
gc.collect()
torch.cuda.empty_cache()

# Set up training with reduced memory usage
training_script = """
# Clone YOLOv5 repo
!git clone https://github.com/ultralytics/yolov5 --depth 1
%cd yolov5

# Install requirements
!pip install -r requirements.txt

# Apply patches to fix memory issues in train.py (addressing the autocast warning)
patch_content = '''
--- train.py
+++ train.py
@@ -409,7 +409,7 @@
         # Forward
         with amp.autocast(enabled=cuda):
-        with torch.cuda.amp.autocast(amp):
+        with torch.amp.autocast('cuda', enabled=amp):
             pred = model(imgs)  # forward
             loss, loss_items = compute_loss(pred, targets.to(device))  # loss scaled by batch_size
             if RANK != -1:
@@ -412,7 +412,7 @@
                 loss = loss * WORLD_SIZE  # gradient averaged between devices in DDP mode
                 if opt.quad:
                     loss *= 4.
-        with torch.cuda.amp.autocast(amp):
+        with torch.amp.autocast('cuda', enabled=amp):
'''

# Create the patch file
with open('fix_autocast.patch', 'w') as f:
    f.write(patch_content)

# Try to apply the patch (may not work perfectly, but will help show what needs changing)
!cat fix_autocast.patch | patch -p0

# Run training with memory-saving options
!python train.py \\
  --img 640 \\
  --batch 8 \\
  --epochs 50 \\
  --data ../license_plate_data.yaml \\
  --weights yolov5s.pt \\
  --cache ram \\
  --workers 2 \\
  --device 0
"""

# Save the training script
with open('train_with_reduced_memory.py', 'w') as f:
    f.write(training_script)

# Provide guidance for manual edits if patching fails
print("""
To manually fix the memory issue:

1. Reduce batch size: Use --batch 8 instead of 16
2. Reduce workers: Use --workers 2 instead of 8
3. Fix the deprecated autocast warnings:
   - Open yolov5/train.py
   - Find instances of: with torch.cuda.amp.autocast(amp):
   - Replace with: with torch.amp.autocast('cuda', enabled=amp):

Run this command with lower memory settings:

!cd yolov5 && python train.py --img 640 --batch 8 --epochs 50 --data ../license_plate_data.yaml --weights yolov5s.pt --cache ram --workers 2
""")

# Create a clean script to run with reduced settings
with open('run_training_lower_memory.py', 'w') as f:
    f.write("""
import os

# Change directory to yolov5 if it exists
if os.path.exists('yolov5'):
    os.chdir('yolov5')
else:
    print("YOLOv5 directory not found. Please run setup first.")
    exit()

# Run training with reduced memory settings
!python train.py --img 640 --batch 8 --epochs 50 --data ../license_plate_data.yaml --weights yolov5s.pt --cache ram --workers 2
""")

print("\nCreated script 'run_training_lower_memory.py' with optimized settings")
print("Run it using: !python run_training_lower_memory.py")


To manually fix the memory issue:

1. Reduce batch size: Use --batch 8 instead of 16
2. Reduce workers: Use --workers 2 instead of 8
3. Fix the deprecated autocast warnings:
   - Open yolov5/train.py
   - Find instances of: with torch.cuda.amp.autocast(amp):
   - Replace with: with torch.amp.autocast('cuda', enabled=amp):

Run this command with lower memory settings:

!cd yolov5 && python train.py --img 640 --batch 8 --epochs 50 --data ../license_plate_data.yaml --weights yolov5s.pt --cache ram --workers 2


Created script 'run_training_lower_memory.py' with optimized settings
Run it using: !python run_training_lower_memory.py


In [6]:
!cd yolov5 && python train.py --img 640 --batch 8 --epochs 100 --data ../license_plate_data.yaml --weights yolov5s.pt --cache ram --workers 2

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
       2/99       2.2G    0.09269    0.09156          0        156        640:  17% 4/23 [00:00<00:02,  7.21it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       2/99       2.2G     0.0915    0.09115          0        175        640:  22% 5/23 [00:00<00:02,  7.62it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
       2/99       2.2G    0.09167    0.08803          0        134        640:  26% 6/23 [00:00<00:02,  7.76it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
   

In [7]:
# Import necessary libraries
import os
import torch
import shutil
from datetime import datetime

# Set the paths
base_dir = "/content/drive/MyDrive/VideoDataset"  # Adjust if your base directory is different
model_dir = f"{base_dir}/models"

# Create models directory if it doesn't exist
os.makedirs(model_dir, exist_ok=True)
print(f"Created directory: {model_dir}")

# The default location where YOLOv5 saves the best trained model
yolo_best_model_path = "yolov5/runs/train/exp/weights/best.pt"  # Adjust if your path is different
yolo_last_model_path = "yolov5/runs/train/exp/weights/last.pt"  # The last checkpoint

# Check if the model exists at the expected location
if not os.path.exists(yolo_best_model_path):
    print(f"Warning: Best model not found at {yolo_best_model_path}")
    print("Searching for models in the training output directory...")

    # Try to find the model in possible alternative locations
    exp_dir = "yolov5/runs/train/"
    found = False

    # Look through all experiment directories
    for exp_folder in sorted(os.listdir(exp_dir), reverse=True):
        weights_dir = os.path.join(exp_dir, exp_folder, "weights")
        if os.path.exists(weights_dir):
            for model_file in os.listdir(weights_dir):
                if model_file == "best.pt":
                    yolo_best_model_path = os.path.join(weights_dir, model_file)
                    found = True
                    print(f"Found best model at: {yolo_best_model_path}")
                    break
        if found:
            break

    if not found:
        print("Best model not found. Will try to use the last checkpoint if available.")
        # Look for last.pt instead
        for exp_folder in sorted(os.listdir(exp_dir), reverse=True):
            weights_dir = os.path.join(exp_dir, exp_folder, "weights")
            if os.path.exists(weights_dir):
                for model_file in os.listdir(weights_dir):
                    if model_file == "last.pt":
                        yolo_last_model_path = os.path.join(weights_dir, model_file)
                        found = True
                        print(f"Found last checkpoint at: {yolo_last_model_path}")
                        break
            if found:
                break

# Get current date for the filename
current_date = datetime.now().strftime("%Y%m%d")

# Define the destination paths with meaningful names
best_model_dest = os.path.join(model_dir, f"license_plate_detector_best_{current_date}.pt")
last_model_dest = os.path.join(model_dir, f"license_plate_detector_last_{current_date}.pt")

# Copy the best model if it exists
if os.path.exists(yolo_best_model_path):
    shutil.copy2(yolo_best_model_path, best_model_dest)
    print(f"Best model saved to: {best_model_dest}")
else:
    print("Best model not found. Skipping...")

# Copy the last checkpoint if it exists
if os.path.exists(yolo_last_model_path):
    shutil.copy2(yolo_last_model_path, last_model_dest)
    print(f"Last checkpoint saved to: {last_model_dest}")
else:
    print("Last checkpoint not found. Skipping...")

# Save a model in TorchScript format for deployment (if best model exists)
if os.path.exists(best_model_dest):
    try:
        # Load the model
        model = torch.hub.load('ultralytics/yolov5', 'custom', path=best_model_dest)

        # Save in TorchScript format for deployment
        torchscript_path = os.path.join(model_dir, f"license_plate_detector_torchscript_{current_date}.pt")
        model.model.eval()  # Set model to evaluation mode
        traced_model = torch.jit.trace(model.model, torch.zeros(1, 3, 640, 640))
        traced_model.save(torchscript_path)
        print(f"TorchScript model saved to: {torchscript_path}")

        # Also export to ONNX format which is useful for many deployment scenarios
        try:
            onnx_path = os.path.join(model_dir, f"license_plate_detector_{current_date}.onnx")
            # Export directly using YOLOv5's export function
            export_cmd = f"cd yolov5 && python export.py --weights {best_model_dest} --include onnx --img 640 --simplify"
            print("Exporting to ONNX format...")
            os.system(export_cmd)

            # The export.py script typically saves the ONNX file in the same directory as the source model
            # We need to copy it to our models directory
            onnx_source = best_model_dest.replace('.pt', '.onnx')
            if os.path.exists(onnx_source):
                shutil.copy2(onnx_source, onnx_path)
                print(f"ONNX model saved to: {onnx_path}")
            else:
                print(f"ONNX export failed: File not found at {onnx_source}")
        except Exception as e:
            print(f"Error exporting to ONNX: {e}")

    except Exception as e:
        print(f"Error saving TorchScript model: {e}")

print("\nModel saving complete.")

# Basic function to demonstrate how to use the saved model
print("\nExample code to use the saved model:")
print("""
# Load and use the model
import torch

# For PyTorch .pt model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path/to/your/license_plate_detector_best.pt')

# For inference
img = 'path/to/test/image.jpg'  # or a PIL Image
results = model(img)

# Display results
results.show()  # display
results.print()  # print results to screen

# Access results data
results.xyxy[0]  # bounding boxes for first image as tensor (x1, y1, x2, y2, confidence, class)
results.pandas().xyxy[0]  # bounding boxes as pandas DataFrame
""")

Created directory: /content/drive/MyDrive/VideoDataset/models
Best model saved to: /content/drive/MyDrive/VideoDataset/models/license_plate_detector_best_20250407.pt
Last checkpoint saved to: /content/drive/MyDrive/VideoDataset/models/license_plate_detector_last_20250407.pt


/usr/local/lib/python3.11/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-4-7 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


Error saving TorchScript model: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

Model saving complete.

Example code to use the saved model:

# Load and use the model
import torch

# For PyTorch .pt model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='path/to/your/license_plate_detector_best.pt')

# For inference
img = 'path/to/test/image.jpg'  # or a PIL Image
results = model(img)

# Display results
results.show()  # display
results.print()  # print results to screen

# Access results data
results.xyxy[0]  # bounding boxes for first image as tensor (x1, y1, x2, y2, confidence, class)
results.pandas().xyxy[0]  # bounding boxes as pandas DataFrame

